In [2]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint
import time

api_key = 'awPgd5YOKYVv0DCC8BERPRPeKJhXmGw4'

docs = []

def get_articles(query, api_key, page=0, filter_params='&facet=true&begin_date=20170101&end_date=20240901'):
    # Adding filter query (fq) for more precise control
    fq = 'fq=body:("Gap") AND body:("clothing") AND NOT body:("Trump") AND NOT body:("Israel")'
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&page={page}&api-key={api_key}&{fq}{filter_params}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()

for i in range(10): # For 100 articles
    print(i)
    articles = get_articles("Gap", api_key, page = i)
    x = articles['response']['docs']
    docs.extend(x)
    time.sleep(12)


    
text = []
for doc in docs:
    text.append(doc['headline']['main'] + '. ' +  doc['abstract'])
text

def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('AFINN-111.txt')
# Scores_Table = read_scores('AFINN-111.txt')

scores = []

for txt in text:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)
!pip install azure-ai-textanalytics

import requests
import azure
import time

key = 'ce14ce8f012d462fbbedd4c70565b678' #"<paste-your-text-analytics-key-here>"
endpoint = 'https://fischba-webdataproject-fall2024.openai.azure.com/' #"<paste-your-text-analytics-endpoint-here>"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client


def sentiment_analysis_example(client, txt):
    documents = [txt]
    response = client.analyze_sentiment(documents=documents)[0]
    print(response)
    responses_all.append(response)

def key_phrase_extraction_example(client, txt):

    try:
        documents = [txt]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            return response.key_phrases
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))

azure_keyphrases = []
azure_sentiments = []
responses_all =[]
txt_number = 0
requests_per_second_limit = 100
time_counter = time.time()  # Start time of processing the request
for txt in text:
    txt_number+=1
    print(txt_number)
    client = authenticate_client()
 # Detect key phrases.
    key_phrases = key_phrase_extraction_example(client,txt)
    azure_keyphrases.append(key_phrases)
 # Detect Sentiments
    sentiments = sentiment_analysis_example(client, txt)

    end_time = time.time()  # End time of processing the request
    processing_time = end_time - time_counter

    requests_in_this_second = txt_number % requests_per_second_limit

    # If we reach the limit of requests per second, add a delay to maintain the limit
    if requests_in_this_second == 0:
        time_taken_for_batch = end_time - time_counter
        if time_taken_for_batch < 1:
            time.sleep(1 - time_taken_for_batch)
        time_counter = time.time()
        
final_sentiments=[]
for i in responses_all:
    x = i['sentiment']
    final_sentiments.append(x)
    
final_sentiments

print(final_sentiments.count('positive'))
print(final_sentiments.count('negative'))

0
1
2
3
4
5
6
7
8
9
1
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.13, neutral=0.85, negative=0.02), 'sentences': [SentenceSentiment(text=The Napoleon of Your Living Room. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.05, neutral=0.94, negative=0.0), length=34, offset=0, mined_opinions=[]), SentenceSentiment(text=Gary Friedman, who runs RH (formerly Restoration Hardware), is out to conquer the world, one luxurious sofa at a time. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.32, neutral=0.63, negative=0.04), length=119, offset=34, mined_opinions=[]), SentenceSentiment(text=Next stop, Waterloo?, sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.02), length=20, offset=153, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
2
{'id': '0', 'sentiment': 'negative', 'warnings': []

{'id': '0', 'sentiment': 'positive', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.53, neutral=0.46, negative=0.0), 'sentences': [SentenceSentiment(text=Gap, Looking for Barbie Magic, Hires Mattel’s President as C.E.O.. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.37, neutral=0.62, negative=0.02), length=67, offset=0, mined_opinions=[]), SentenceSentiment(text=Richard Dickson helped improve Barbie’s image, culminating in the blockbuster film. , sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=0.53, neutral=0.46, negative=0.0), length=84, offset=67, mined_opinions=[]), SentenceSentiment(text=Now he faces a similar turnaround challenge at the retailer., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.91, negative=0.07), length=60, offset=151, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
10
{'id': '0', 'sentiment': 'neutral'

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.06, neutral=0.87, negative=0.07), 'sentences': [SentenceSentiment(text=How PETA Won Its Messy Fight and Took a Seat at the Table. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.89, negative=0.09), length=59, offset=0, mined_opinions=[]), SentenceSentiment(text=Turns out if you throw enough red paint, eventually they’ll listen., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.1, neutral=0.86, negative=0.04), length=67, offset=59, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
20
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.69, negative=0.31), 'sentences': [SentenceSentiment(text=Protests in Bangladesh Shake a Global Workshop for Apparel. , sentiment=neutral, confidence_scor

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), 'sentences': [SentenceSentiment(text=Paul Smith on How to Sew Patches in a Traditional Japanese Stitch. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), length=67, offset=0, mined_opinions=[]), SentenceSentiment(text=In the latest installment of our Designer D.I.Y. series, learn how to breathe new life into old clothes with a classic sashiko stitch., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.0), length=134, offset=67, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
30
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.88, negative=0.1), 'sentences': [SentenceSentiment(text=Inspired or Frustrated

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.86, negative=0.14), 'sentences': [SentenceSentiment(text=Adidas Ends Partnership With Kanye West at a Considerable Cost. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.0), length=64, offset=0, mined_opinions=[]), SentenceSentiment(text=The German sportswear giant is the latest company to cut ties with Mr. West, now known as Ye, after his antisemitic remarks. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.01), length=125, offset=64, mined_opinions=[]), SentenceSentiment(text=Ending the partnership is expected to hurt company profits., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.6, negative=0.4), length=59, offset=189, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
41


...

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.47, negative=0.52), 'sentences': [SentenceSentiment(text=What to See in New York Art Galleries This Week. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), length=49, offset=0, mined_opinions=[]), SentenceSentiment(text=Ralph Humphrey’s monochrome paintings; , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.98, negative=0.01), length=39, offset=49, mined_opinions=[]), SentenceSentiment(text=Ivy Haldeman‘s anthropomorphic hot dogs; , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.04, neutral=0.96, negative=0.01), length=41, offset=88, mined_opinions=[]), SentenceSentiment(text=Trudy Benson’s rectangular grids; , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.05, neutral=0.92, negative=0.04), len

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.48, negative=0.52), 'sentences': [SentenceSentiment(text=Behind the Story: The Case That Came Back to Haunt a San Francisco Judge. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.91, negative=0.09), length=74, offset=0, mined_opinions=[]), SentenceSentiment(text=After 40 years, Anthony Kline had to confront a case he had forgotten., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.48, negative=0.52), length=70, offset=74, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
71
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.15, negative=0.84), 'sentences': [SentenceSentiment(text=Profits From Store-Branded Credit Cards Hide Depth of Retailers’ Troubles. , 

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.76, negative=0.22), 'sentences': [SentenceSentiment(text=Paid Time Off, Free Fries: How Corporate America Is Getting Out the Vote. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.94, negative=0.03), length=74, offset=0, mined_opinions=[]), SentenceSentiment(text=Spurred by recent social unrest, many U.S. companies are trying to make it easier for workers, and urging their customers, to cast their ballots., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.59, negative=0.4), length=145, offset=74, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
81
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.01), 'sentences': [SentenceSentiment(text=T

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.91, negative=0.08), 'sentences': [SentenceSentiment(text=Is Pants Sizing Sexist?. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.93, negative=0.06), length=25, offset=0, mined_opinions=[]), SentenceSentiment(text=Our critic examines the history of women’s trousers and “the sheer ridiculousness of the current state of sizing.”, sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.88, negative=0.11), length=114, offset=25, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
91
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.13, neutral=0.87, negative=0.0), 'sentences': [SentenceSentiment(text=Home Cooks Find Bargains at the Restaurant Supply Store. , sentiment=neutral, conf

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.91, negative=0.08), 'sentences': [SentenceSentiment(text=Australian Government Buys Copyright to Indigenous Flag. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.0), length=57, offset=0, mined_opinions=[]), SentenceSentiment(text=The flag had been at the center of a dispute, but the move allows anyone to reproduce its design without seeking permission or paying a fee., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.83, negative=0.16), length=140, offset=57, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
8
24
